In [5]:
import numpy as np 
import pandas as pd 
import re    #Regular Expressions ( re ) module in Python
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

In [6]:
data = pd.read_csv('/content/Sentiment.csv')

In [7]:
data=data[['text','sentiment']]

In [8]:
data

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [9]:
data = data[data.sentiment != "Neutral"]
data

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [10]:
data['text'] = data['text'].apply(lambda x: x.lower())
data


<ipython-input-10-165dfba0cd9a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())


,text,sentiment
1,rt @scottwalker: didn't catch the full #gopdeb...,Positive
3,rt @robgeorge: that carly fiorina is trending ...,Positive
4,rt @danscavino: #gopdebate w/ @realdonaldtrump...,Positive
5,"rt @gregabbott_tx: @tedcruz: ""on my first day ...",Positive
6,rt @warriorwoman91: i liked her and was happy ...,Negative
...,...,...
13866,rt @cappy_yarbrough: love to see men who will ...,Negative
13867,rt @georgehenryw: who thought huckabee exceede...,Positive
13868,"rt @lrihendry: #tedcruz as president, i will a...",Positive
13869,rt @jrehling: #gopdebate donald trump says tha...,Negative


In [11]:
data['text'] = data['text'].apply((lambda x: re.sub('[^a-z0-9\s]','',x))) #The syntax for re. sub() is re. sub(pattern,repl,string).
data

<ipython-input-11-107da71e7f58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-z0-9\s]','',x))) #The syntax for re. sub() is re. sub(pattern,repl,string).


,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbotttx tedcruz on my first day i will...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative
...,...,...
13866,rt cappyyarbrough love to see men who will nev...,Negative
13867,rt georgehenryw who thought huckabee exceeded ...,Positive
13868,rt lrihendry tedcruz as president i will alway...,Positive
13869,rt jrehling gopdebate donald trump says that h...,Negative


In [12]:
data.loc[1]

text         rt scottwalker didnt catch the full gopdebate ...
sentiment                                             Positive
Name: 1, dtype: object

In [14]:
for index,row in data.iterrows():
      row[0] = row[0].replace('rt ',' ')
data

,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive
5,gregabbotttx tedcruz on my first day i will r...,Positive
6,warriorwoman91 i liked her and was happy when...,Negative
...,...,...
13866,cappyyarbrough love to see men who will never...,Negative
13867,georgehenryw who thought huckabee exceeded th...,Positive
13868,lrihendry tedcruz as president i will always ...,Positive
13869,jrehling gopdebate donald trump says that he ...,Negative


In [15]:
data[0:2]

,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive


In [16]:
data['text'].values[1]

' robgeorge that carly fiorina is trending  hours after her debate  above any of the men in justcompleted gopdebate says shes on '

In [17]:
data['text'].values[10]

' pattonoswalt i loved scott walker as mark harmons romantic rival in summer school look it up gopdebate'

In [18]:
data['sentiment'].values[10]

'Positive'

In [19]:
data[ data['sentiment'] == 'Positive']
# data.loc[data['sentiment'] == 'Positive']

,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive
5,gregabbotttx tedcruz on my first day i will r...,Positive
11,waynedupreeshow just woke up to tweet this ou...,Positive
...,...,...
13859,best line of gopdebate was immigration without...,Positive
13862,rwsurfergirl trump has got it right nobody wo...,Positive
13867,georgehenryw who thought huckabee exceeded th...,Positive
13868,lrihendry tedcruz as president i will always ...,Positive


In [20]:
len(data[ data['sentiment'] == 'Positive'])

2236

In [21]:
data[ data['sentiment'] == 'Negative']

,text,sentiment
6,warriorwoman91 i liked her and was happy when...,Negative
8,deer in the headlights lizzwinstead ben carso...,Negative
9,nancyosborne180 last nights debate proved it ...,Negative
10,jgreendc realdonaldtrump in all fairness billc...,Negative
12,me reading my familys comments about how great...,Negative
...,...,...
13863,so trans soldiers can die for you huckabee but...,Negative
13864,rwsurfergirl is it just me or does anyone els...,Negative
13865,rwsurfergirl fox is cherry picking the candid...,Negative
13866,cappyyarbrough love to see men who will never...,Negative


In [22]:
len(data[ data['sentiment'] == 'Negative'])

8493

In [23]:
data['text'].values[0:3]

array([' scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff',
       ' robgeorge that carly fiorina is trending  hours after her debate  above any of the men in justcompleted gopdebate says shes on ',
       ' danscavino gopdebate w realdonaldtrump delivered the highest ratings in the history of presidential debates trump2016 httptco'],
      dtype=object)

In [24]:
top_words = 2000
tokenizer = Tokenizer(num_words=top_words, split=' ')
tokenizer.fit_on_texts(data['text'].values) 

In [25]:
word_index =  tokenizer.word_index
word_index

{'the': 1,
 'gopdebate': 2,
 'gopdebates': 3,
 'to': 4,
 'is': 5,
 'of': 6,
 'a': 7,
 'and': 8,
 'i': 9,
 'rwsurfergirl': 10,
 'you': 11,
 'in': 12,
 'it': 13,
 'for': 14,
 'trump': 15,
 'that': 16,
 'on': 17,
 'fox': 18,
 'not': 19,
 'this': 20,
 'was': 21,
 'about': 22,
 'realdonaldtrump': 23,
 'amp': 24,
 'news': 25,
 'debate': 26,
 'from': 27,
 'have': 28,
 'at': 29,
 'we': 30,
 'me': 31,
 'but': 32,
 'he': 33,
 'be': 34,
 'candidates': 35,
 'are': 36,
 'just': 37,
 'like': 38,
 'last': 39,
 'they': 40,
 'so': 41,
 'gop': 42,
 'dont': 43,
 'megynkelly': 44,
 'with': 45,
 'all': 46,
 'up': 47,
 'what': 48,
 'people': 49,
 'jeb': 50,
 'foxnews': 51,
 'who': 52,
 'my': 53,
 'how': 54,
 'as': 55,
 'if': 56,
 'bush': 57,
 'night': 58,
 'has': 59,
 'no': 60,
 'their': 61,
 'by': 62,
 'when': 63,
 'can': 64,
 'do': 65,
 'one': 66,
 'more': 67,
 'im': 68,
 'out': 69,
 'or': 70,
 'would': 71,
 'his': 72,
 'him': 73,
 'chris': 74,
 'get': 75,
 'think': 76,
 'republican': 77,
 'donald': 78,
 

In [26]:
len(word_index)

15713

In [27]:
X = tokenizer.texts_to_sequences(data['text'].values)
print(len(X))
print(X[0])


10729
[358, 120, 1, 712, 2, 39, 58, 234, 36, 207, 6, 172, 1741, 12, 1302, 1387, 732]


In [28]:
# X

In [29]:
len(X[0])

17

In [30]:
len(X[10728])

11

In [31]:
data['text'].values[0]

' scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff'

In [32]:
word_index['scottwalker']

358

In [33]:
word_index['didnt']

120

In [34]:
word_index['catch']

2370

In [35]:
word_index['the']

1

In [36]:
word_index['httptcozsff']

6321

In [37]:
data['text'][0:2]

1     scottwalker didnt catch the full gopdebate la...
3     robgeorge that carly fiorina is trending  hou...
Name: text, dtype: object

In [38]:
data['sentiment'][0:2]

1    Positive
3    Positive
Name: sentiment, dtype: object

In [39]:
X = pad_sequences(X)
print(X[0])


[   0    0    0    0    0    0    0    0    0    0    0  358  120    1
  712    2   39   58  234   36  207    6  172 1741   12 1302 1387  732]


In [40]:
len(X[0])

28

In [41]:
# X = sequence.pad_sequences(X, maxlen=30)
# X[0]

In [42]:
X.shape

(10729, 28)

In [43]:
X.shape[1]

28

In [44]:
embed_dim = 128
lstm_units = 196

model = Sequential()
model.add(Embedding(top_words, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_units))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 1)                 197       
                                                                 
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
parameters_of_Embedding_layer=top_words*embed_dim
parameters_of_Embedding_layer

256000

In [46]:
parameters_of_lstm_layer=(embed_dim+lstm_units+1)*lstm_units*4
parameters_of_lstm_layer

254800

In [47]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y=le.fit_transform(data['sentiment'])
print(le.classes_)
print(y)

['Negative' 'Positive']
[1 1 1 ... 1 0 1]


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(7188, 28) (7188,)
(3541, 28) (3541,)


In [49]:
batch_size = 32
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size,validation_data= (X_test, y_test)
                             ,callbacks=EarlyStopping(patience=3,restore_best_weights=True))

Epoch 1/7
225/225 [==============================] - 20s 77ms/step - loss: 0.4226 - accuracy: 0.8197 - val_loss: 0.3437 - val_accuracy: 0.8515
Epoch 2/7
225/225 [==============================] - 16s 73ms/step - loss: 0.3014 - accuracy: 0.8723 - val_loss: 0.3337 - val_accuracy: 0.8628
Epoch 3/7
225/225 [==============================] - 16s 72ms/step - loss: 0.2495 - accuracy: 0.9007 - val_loss: 0.3743 - val_accuracy: 0.8382
Epoch 4/7
225/225 [==============================] - 16s 73ms/step - loss: 0.2087 - accuracy: 0.9140 - val_loss: 0.3997 - val_accuracy: 0.8472
Epoch 5/7
225/225 [==============================] - 17s 74ms/step - loss: 0.1787 - accuracy: 0.9302 - val_loss: 0.4252 - val_accuracy: 0.8435


In [50]:
model.evaluate(X_train, y_train)

225/225 [==============================] - 4s 18ms/step - loss: 0.2327 - accuracy: 0.9104


[0.23272907733917236, 0.910406231880188]

In [51]:
model.evaluate(X_test, y_test)

111/111 [==============================] - 2s 18ms/step - loss: 0.3337 - accuracy: 0.8628


[0.3336893320083618, 0.8627506494522095]

In [52]:
import numpy as np
y_pred=(model.predict(X_test)>=0.5).astype(int)
y_pred

111/111 [==============================] - 3s 21ms/step


array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [53]:
data['text'].values[10]

' pattonoswalt i loved scott walker as mark harmons romantic rival in summer school look it up gopdebate'

In [54]:
data['sentiment'].values[10]

'Positive'

In [55]:
twt = ['pattonoswalt i loved scott walker as mark harmons romantic rival in summer school look it up gopdebate	'	]

twt = tokenizer.texts_to_sequences(twt)

twt


[[660, 9, 519, 301, 165, 55, 12, 1305, 180, 13, 47, 2]]

In [72]:
print(word_index['pattonoswalt'])
print(word_index['i'])
print(word_index['loved'])
print(word_index['mark'])
print(word_index['harmons'])
print(word_index['romantic'])

660
9
519
2372
6332
6333


In [57]:
twt = pad_sequences(twt, maxlen=28)
print(twt)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0  660    9  519  301  165   55   12 1305  180   13   47    2]]


In [58]:
twt.shape

(1, 28)

In [59]:
print(model.predict(twt))

1/1 [==============================] - 0s 29ms/step
[[0.7018799]]


In [60]:
prediction_twt=y_pred=(model.predict(twt)>=0.5).astype(int)
prediction_twt

1/1 [==============================] - 0s 27ms/step


array([[1]])

In [61]:
X_test[1000].shape

(28,)

In [62]:
X_test_reshape=X_test[1000].reshape(1,-1)
X_test_reshape.shape

(1, 28)

In [63]:
(model.predict(X_test_reshape)>=0.5).astype(int)

1/1 [==============================] - 0s 42ms/step


array([[0]])

In [64]:
y_test[1000]

0

In [65]:
y_test[1000]

0